In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit,col,instr,expr,pow,round,bround,corr,monotonically_increasing_id,initcap,lower,lpad,rpad,regexp_extract,regexp_replace,translate,locate,current_date,current_timestamp,date_add,date_sub,coalesce
import os

In [3]:
spark = SparkSession.builder.appName('df2').getOrCreate()

In [4]:
datapath = '/home/amogh/Documents/spark_certification/Spark-The-Definitive-Guide-master/data/'

In [5]:
df = spark.read.\
    option('header',True).\
        option('inferschema',True).\
            csv(datapath+'retail-data/by-day/2010-12-01.csv').sample(fraction=0.1)

In [6]:
df.printSchema()
df.createOrReplaceTempView("dfTable")

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [7]:
# convert to sparktype

In [9]:
df.select(lit(5),lit("five"))

DataFrame[5: int, five: string]

In [11]:
# booleans

In [16]:
df.where( col("InvoiceNo") == (536365) ).\
    select("InvoiceNo", "Description").show(5,False)

+---------+---------------------------------+
|InvoiceNo|Description                      |
+---------+---------------------------------+
|536365   |GLASS STAR FROSTED T-LIGHT HOLDER|
+---------+---------------------------------+



In [23]:
pricefilter = col("UnitPrice") > 600
decfilter = instr(col('Description'),"POSTAGE") >= 1

df.where(col('StockCode').isin("DOT") ).where((pricefilter |decfilter  ) ).show(3)

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



In [24]:
# we can also use it as boolean column

In [25]:
df.withColumn("isExpensive",(pricefilter |decfilter  ) ).where('isExpensive').show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|isExpensive|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----------+
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|       true|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----------+



In [27]:
df.withColumn("isExpensive", expr("NOT UnitPrice <= 250"))\
.where("isExpensive")\
.select("Description", "UnitPrice").show(5)

+--------------+---------+
|   Description|UnitPrice|
+--------------+---------+
|DOTCOM POSTAGE|   607.49|
+--------------+---------+



In [28]:
# Numerical column

In [29]:
fabricatedQuantity = pow(col("Quantity") * col("UnitPrice"),2) + 5

In [31]:
df.select(expr("CustomerId"),fabricatedQuantity.alias("realQuantity")).show()

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|            655.25|
|   13047.0|103.00999999999998|
|   13047.0|401.00999999999993|
|   17850.0| 887.0900000000001|
|   17850.0|            103.01|
|   17850.0|          418.7156|
|   17850.0|45.449600000000004|
|   17850.0|45.449600000000004|
|   17850.0|163.76000000000005|
|   17850.0|239.08999999999997|
|   14688.0|             77.25|
|   14688.0|            277.25|
|   14688.0|            385.25|
|   15311.0|         9560.0625|
|   15311.0|           13.7025|
|   16098.0| 397.0399999999999|
|   16098.0|             446.0|
|   16029.0| 537939.2335999999|
|   16029.0|          291605.0|
|   16250.0|323.62250000000006|
+----------+------------------+
only showing top 20 rows



In [32]:
# we can use round

In [36]:
df.selectExpr("CustomerId", "round(power((Quantity * UnitPrice),2.0) + 5,2) as realQuantity").show()

+----------+------------+
|CustomerId|realQuantity|
+----------+------------+
|   17850.0|      655.25|
|   13047.0|      103.01|
|   13047.0|      401.01|
|   17850.0|      887.09|
|   17850.0|      103.01|
|   17850.0|      418.72|
|   17850.0|       45.45|
|   17850.0|       45.45|
|   17850.0|      163.76|
|   17850.0|      239.09|
|   14688.0|       77.25|
|   14688.0|      277.25|
|   14688.0|      385.25|
|   15311.0|     9560.06|
|   15311.0|        13.7|
|   16098.0|      397.04|
|   16098.0|       446.0|
|   16029.0|   537939.23|
|   16029.0|    291605.0|
|   16250.0|      323.62|
+----------+------------+
only showing top 20 rows



In [39]:
# using round 

df.select(round(lit("2.5")), bround(lit("2.5"))).show(2)

+-------------+--------------+
|round(2.5, 0)|bround(2.5, 0)|
+-------------+--------------+
|          3.0|           2.0|
|          3.0|           2.0|
+-------------+--------------+
only showing top 2 rows



In [40]:
# working with stats

In [41]:
df.stat.corr("Quantity", "UnitPrice")

-0.022578777066752327

In [43]:
df.select(corr("Quantity", "UnitPrice")).show()

+-------------------------+
|corr(Quantity, UnitPrice)|
+-------------------------+
|     -0.02257877706675...|
+-------------------------+



In [44]:
df.describe()

DataFrame[summary: string, InvoiceNo: string, StockCode: string, Description: string, Quantity: string, UnitPrice: string, CustomerID: string, Country: string]

In [47]:
# approxquantile

df.stat.approxQuantile("UnitPrice",[0.5],0)


[2.51]

In [48]:
df.stat.crosstab("StockCode", "Quantity").show()

+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|StockCode_Quantity| -7|  1| 10| 12| 16| 18|192|  2| 23| 24| 27|  3| 32| 36|  4| 40|432| 48|  5|  6|  7|  8| 96|
+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|             22219|  0|  0|  0|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|
|             72817|  0|  0|  0|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|
|             22988|  0|  0|  0|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  1|  0|  0|
|             22379|  0|  1|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|
|             22585|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  1|  0|  0|  0|  0|
|            85114C|  0|  0|  0|  0|  0|  0|  0|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|

In [49]:
df.stat.freqItems(["StockCode", "Quantity"]).show()

+--------------------+--------------------+
| StockCode_freqItems|  Quantity_freqItems|
+--------------------+--------------------+
|[22620, 84029G, 7...|[23, 32, 8, 2, -7...|
+--------------------+--------------------+



In [52]:
df.select(monotonically_increasing_id()).show()

+-----------------------------+
|monotonically_increasing_id()|
+-----------------------------+
|                            0|
|                            1|
|                            2|
|                            3|
|                            4|
|                            5|
|                            6|
|                            7|
|                            8|
|                            9|
|                           10|
|                           11|
|                           12|
|                           13|
|                           14|
|                           15|
|                           16|
|                           17|
|                           18|
|                           19|
+-----------------------------+
only showing top 20 rows



In [53]:
# String functions

In [55]:
df.select(initcap(col("Description"))).show(2,False)

+---------------------------------+
|initcap(Description)             |
+---------------------------------+
|Glass Star Frosted T-light Holder|
|Ivory Knitted Mug Cosy           |
+---------------------------------+
only showing top 2 rows



In [57]:
df.select(lower(col("Description"))).show(2,False)


+---------------------------------+
|lower(Description)               |
+---------------------------------+
|glass star frosted t-light holder|
|ivory knitted mug cosy           |
+---------------------------------+
only showing top 2 rows



In [83]:
df.select(lpad(lit("HELLO"),3," ")).show(4)

+-----------------+
|lpad(HELLO, 3,  )|
+-----------------+
|              HEL|
|              HEL|
|              HEL|
|              HEL|
+-----------------+
only showing top 4 rows



In [80]:
df.select(rpad(lit("HELLO"),10,"l")).show(4)

+------------------+
|rpad(HELLO, 10, l)|
+------------------+
|        HELLOlllll|
|        HELLOlllll|
|        HELLOlllll|
|        HELLOlllll|
+------------------+
only showing top 4 rows



In [3]:
# Regular expressions

In [21]:
regex_string = "BLACK|WHITE|RED|GREEN|BLUE"

In [22]:
df.select(regexp_replace(col("Description"), regex_string, "COLOR").alias("color_clean"),
col("Description")).show(2)

+--------------------+--------------------+
|         color_clean|         Description|
+--------------------+--------------------+
| COLOR METAL LANTERN| WHITE METAL LANTERN|
|KNITTED UNION FLA...|KNITTED UNION FLA...|
+--------------------+--------------------+
only showing top 2 rows



In [12]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [24]:
df.select(translate(col("Description"),'LEET','1337'),col("Description")).show()

+----------------------------------+--------------------+
|translate(Description, LEET, 1337)|         Description|
+----------------------------------+--------------------+
|               WHI73 M37A1 1AN73RN| WHITE METAL LANTERN|
|              KNI773D UNION F1A...|KNITTED UNION FLA...|
|              BA7H BUI1DING B1O...|BATH BUILDING BLO...|
|              PAP3R CHAIN KI7 5...|PAPER CHAIN KIT 5...|
|              3DWARDIAN PARASO1...|EDWARDIAN PARASOL...|
|              WOOD 2 DRAW3R CAB...|WOOD 2 DRAWER CAB...|
|              R3D WOO11Y HO77I3...|RED WOOLLY HOTTIE...|
|               WHI73 M37A1 1AN73RN| WHITE METAL LANTERN|
|              KNI773D UNION F1A...|KNITTED UNION FLA...|
|              1UNCH BAG R3D R37...|LUNCH BAG RED RET...|
|              1UNCH BOX WI7H CU...|LUNCH BOX WITH CU...|
|              PACK OF 72 R37ROS...|PACK OF 72 RETROS...|
|               GIR1Y PINK 7OO1 S37| GIRLY PINK TOOL SET|
|              JUMBO SHOPP3R VIN...|JUMBO SHOPPER VIN...|
|             

In [25]:
# regex extract

In [26]:
extract_str = "(BLACK|WHITE|RED|GREEN|BLUE)"

In [27]:
df.select(
regexp_extract(col("Description"), extract_str, 1).alias("color_clean"),
col("Description")).show(2)

+-----------+--------------------+
|color_clean|         Description|
+-----------+--------------------+
|      WHITE| WHITE METAL LANTERN|
|           |KNITTED UNION FLA...|
+-----------+--------------------+
only showing top 2 rows



In [28]:
# we may want to check if value exists

In [29]:
containes_white = instr(col('Description'),'WHITE') >= 1

In [ ]:
df.withColumn()

In [30]:
df.withColumn("hasSimpleColor", containes_white)\
.where("hasSimpleColor")\
.select("Description").show(3, False)

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE METAL LANTERN               |
|WOOD 2 DRAWER CABINET WHITE FINISH|
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



In [37]:
simple_colors = ["black", "white", "red", "green", "blue"]

In [47]:
[c for c in simple_colors]

['black', 'white', 'red', 'green', 'blue']

In [51]:
def color_locator(column, color_string):
    return locate(color_string.upper(), column)\
    .cast("boolean")\
    .alias("is_" + color_string)

In [58]:
selectedColumns = [color_locator(column = df.Description,color_string = c) for c in simple_colors ]
selectedColumns.append(expr("*"))
df.select(*selectedColumns).where(expr("is_white OR is_red"))\
.select("Description").show(3, False)


+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE METAL LANTERN               |
|EDWARDIAN PARASOL RED             |
|WOOD 2 DRAWER CABINET WHITE FINISH|
+----------------------------------+
only showing top 3 rows



[Column<b'CAST(locate(BLACK, Description, 1) AS BOOLEAN) AS `is_black`'>,
 Column<b'CAST(locate(WHITE, Description, 1) AS BOOLEAN) AS `is_white`'>,
 Column<b'CAST(locate(RED, Description, 1) AS BOOLEAN) AS `is_red`'>,
 Column<b'CAST(locate(GREEN, Description, 1) AS BOOLEAN) AS `is_green`'>,
 Column<b'CAST(locate(BLUE, Description, 1) AS BOOLEAN) AS `is_blue`'>,
 Column<b'unresolvedstar()'>]

In [66]:
# UDFs 

def power1(number): 
    Double = number * number 

    return Double

def power3(double_value):
    return double_value ** 3
    

In [67]:
power3(2)

8

In [75]:
udf_add = spark.udf.register('app',power1)

In [72]:
df.select(udf_add("Quantity")).show()

+-------------+
|app(Quantity)|
+-------------+
|           36|
|           36|
|            4|
|           36|
|            4|
|           36|
|           36|
|           36|
|           36|
|           36|
|          100|
|          100|
|          100|
|          529|
|            1|
|          144|
|          100|
|        36864|
|       186624|
|            9|
+-------------+
only showing top 20 rows



In [59]:
# Datetime

In [6]:
datedf = spark.range(10).\
    withColumn('today',current_date())\
        .withColumn('now',current_timestamp())

In [7]:
datedf.printSchema()

root
 |-- id: long (nullable = false)
 |-- today: date (nullable = false)
 |-- now: timestamp (nullable = false)



In [10]:
datedf.select(date_sub(col('today'),5)).show()

+------------------+
|date_sub(today, 5)|
+------------------+
|        2020-09-05|
|        2020-09-05|
|        2020-09-05|
|        2020-09-05|
|        2020-09-05|
|        2020-09-05|
|        2020-09-05|
|        2020-09-05|
|        2020-09-05|
|        2020-09-05|
+------------------+



In [11]:
from pyspark.sql.functions import datediff, months_between, to_date

In [15]:
datedf.withColumn('week_ago',date_sub('today',7)).select(datediff(col('week_ago'),col('today'))).show()

+-------------------------+
|datediff(week_ago, today)|
+-------------------------+
|                       -7|
|                       -7|
|                       -7|
|                       -7|
|                       -7|
|                       -7|
|                       -7|
|                       -7|
|                       -7|
|                       -7|
+-------------------------+



In [16]:
# date format
date_format = "yyyy-dd-MM"

In [19]:
spark.range(1).select(to_date(lit("2017-12-11"),date_format).alias("date"),to_date(lit("2017-20-12"), date_format).alias("date2")).show()

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+



In [ ]:
# Null values

In [21]:
df.select(coalesce(col("Description"), col("CustomerId"))).show()

+---------------------------------+
|coalesce(Description, CustomerId)|
+---------------------------------+
|             HAND WARMER UNION...|
|             ASSORTED COLOUR B...|
|             FELTCRAFT PRINCES...|
|             ROUND SNACK BOXES...|
|             HAND WARMER UNION...|
|             WOOD S/3 CABINET ...|
|             VICTORIAN SEWING ...|
|             WOOD 2 DRAWER CAB...|
|             HOT WATER BOTTLE ...|
|             AIRLINE LOUNGE,ME...|
|             CLEAR DRAWER KNOB...|
|             FELT EGG COSY CHI...|
|             YOU'RE CONFUSING ...|
|             YELLOW BREAKFAST ...|
|             HANGING MEDINA LA...|
|             ENAMEL FLOWER JUG...|
|             FAIRY TALE COTTAG...|
|             ROSE CARAVAN DOOR...|
|             JAM JAR WITH PINK...|
|             ROSE COTTAGE KEEP...|
+---------------------------------+
only showing top 20 rows



In [22]:
df.na.drop()

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: timestamp, UnitPrice: double, CustomerID: double, Country: string]

In [23]:
fill_cols_vals = {"StockCode": 5, "Description" : "No Value"}
df.na.fill(fill_cols_vals)

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: timestamp, UnitPrice: double, CustomerID: double, Country: string]

In [24]:
df.na.replace([""],["unknown"],"Description")

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: timestamp, UnitPrice: double, CustomerID: double, Country: string]

In [25]:
# working with complext data types

In [26]:
from pyspark.sql.functions import struct
complexDF = df.select(struct("Description", "InvoiceNo").alias("complex"))

In [29]:
complexDF.select("complex.Description").show()

+--------------------+
|         Description|
+--------------------+
|HAND WARMER UNION...|
|ASSORTED COLOUR B...|
|FELTCRAFT PRINCES...|
|ROUND SNACK BOXES...|
|HAND WARMER UNION...|
|WOOD S/3 CABINET ...|
|VICTORIAN SEWING ...|
|WOOD 2 DRAWER CAB...|
|HOT WATER BOTTLE ...|
|AIRLINE LOUNGE,ME...|
|CLEAR DRAWER KNOB...|
|FELT EGG COSY CHI...|
|YOU'RE CONFUSING ...|
|YELLOW BREAKFAST ...|
|HANGING MEDINA LA...|
|ENAMEL FLOWER JUG...|
|FAIRY TALE COTTAG...|
|ROSE CARAVAN DOOR...|
|JAM JAR WITH PINK...|
|ROSE COTTAGE KEEP...|
+--------------------+
only showing top 20 rows



In [7]:
from pyspark.sql.functions import split
df.select(split(col('Description')," ").alias('array_col')).selectExpr("array_col[0]").show(2)


+------------+
|array_col[0]|
+------------+
|        HAND|
|     POPPY'S|
+------------+
only showing top 2 rows



In [9]:
from pyspark.sql.functions import size,array_contains
df.select(size(split(col("Description"), " "))).show(2)

+---------------------------+
|size(split(Description,  ))|
+---------------------------+
|                          5|
|                          3|
+---------------------------+
only showing top 2 rows



In [10]:
df.select(array_contains(split(col("Description"), " "),'WHITE'))

DataFrame[array_contains(split(Description,  ), WHITE): boolean]

In [12]:
from pyspark.sql.functions import split, explode
df.withColumn("splitted", split(col("Description"), " "))\
.withColumn("exploded", explode(col("splitted")))\
.select("Description", "InvoiceNo", "exploded").show(2)

+--------------------+---------+--------+
|         Description|InvoiceNo|exploded|
+--------------------+---------+--------+
|HAND WARMER RED P...|   536366|    HAND|
|HAND WARMER RED P...|   536366|  WARMER|
+--------------------+---------+--------+
only showing top 2 rows



In [13]:
# map

In [14]:
from pyspark.sql.functions import create_map
df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map"))\
.show(2)

+--------------------+
|         complex_map|
+--------------------+
|[HAND WARMER RED ...|
|[POPPY'S PLAYHOUS...|
+--------------------+
only showing top 2 rows



In [16]:
# json

In [17]:
jsonDF = spark.range(1).selectExpr("""'{"myJSONKey" : {"myJSONValue" : [1, 2, 3]}}' as jsonString""")

In [21]:
from pyspark.sql.functions import get_json_object, json_tuple

jsonDF.select(
get_json_object(col("jsonString"), "$.myJSONKey.myJSONValue[1]") ,
json_tuple(col("jsonString"), "myJSONKey")).show(2)

+-------------------------------------------------------+--------------------+
|get_json_object(jsonString, $.myJSONKey.myJSONValue[1])|                  c0|
+-------------------------------------------------------+--------------------+
|                                                      2|{"myJSONValue":[1...|
+-------------------------------------------------------+--------------------+



In [22]:
# to_json and from_json